# 9.2 Types of Joins

In the previous section, we discussed how to _merge_ (or _join_) two data sets by matching on certain variables. But what happens when no match can be found for a row in one `DataFrame`? 

First, let's determine how _pandas_ handles this situation by default. The name "Nevaeh", which is "Heaven" spelled backwards, is said to have taken off when Sonny Sandoval of the band P.O.D. gave his daughter the name in 2000. Let's look at how common this name was four years earlier and four years after.

In [1]:
import pandas as pd
pd.options.display.max_rows = 8

names1996 = pd.read_csv("http://github.com/dlsun/data-science-book/blob/"
                        "master/data/names/yob1996.txt?raw=true",
                        header=None,
                        names=["Name", "Sex", "Count"])
names2004 = pd.read_csv("http://github.com/dlsun/data-science-book/blob/"
                        "master/data/names/yob2004.txt?raw=true",
                        header=None,
                        names=["Name", "Sex", "Count"])

In [2]:
names1996[names1996.Name == "Nevaeh"]

,Name,Sex,Count


In [3]:
names2004[names2004.Name == "Nevaeh"]

,Name,Sex,Count
103,Nevaeh,F,3179
21758,Nevaeh,M,35


In 1996, there were no girls (or fewer than 5) named Nevaeh; just eight years later, there were over 3000 girls (and 27 boys) with the name. It seems like Sonny Sandoval had a huge effect.

What will happen to the name "Nevaeh" when we merge the two data sets?

In [4]:
names = names1996.merge(names2004, on=["Name", "Sex"])
names[names.Name == "Nevaeh"]

,Name,Sex,Count_x,Count_y


By default, _pandas_ only includes combinations that are present in _both_ `DataFrame`s. If it cannot find a match for a row in one `DataFrame`, then the combination is simply dropped.

But in this context, the fact that a name does not appear in one data set is informative. It means that no babies were born in that year with that name. (Technically, it means that fewer than 5 babies were born with that name, as any name that was assigned fewer than 5 times is omitted for privacy reasons.) We might want to include names that appeared in only one of the two `DataFrame`s, rather than just the names that appeared in both. 

There are four types of joins, distinguished by whether they include names from the left `DataFrame`, the right `DataFrame`, both, or neither:

1. **inner join** (default): only values that are present in _both_ `DataFrame`s are included in the result
2. **outer join**: any value that appears in _either_ `DataFrame` is included in the result
3. **left join**: any value that appears in the _left_ `DataFrame` is included in the result, whether or not it appears in the right `DataFrame`
4. **right join**: any value that appears in the _right_ `DataFrame` is included in the result, whether or not it appears in the left `DataFrame`.

In _pandas_, the join type is specified using the `how=` argument.

Now let's look at examples of each of these types of joins.

In [5]:
# inner join
names_inner = names1996.merge(names2004, on=["Name", "Sex"], how="inner")
names_inner

,Name,Sex,Count_x,Count_y
0,Emily,F,25150,25028
1,Jessica,F,24192,9469
2,Ashley,F,23676,14370
3,Sarah,F,21029,12732
...,...,...,...,...
20049,Zhane,M,5,15
20050,Zhi,M,5,9
20051,Zoran,M,5,14
20052,Zyler,M,5,23


In [6]:
# outer join
names_outer = names1996.merge(names2004, on=["Name", "Sex"], how="outer")
names_outer

,Name,Sex,Count_x,Count_y
0,Emily,F,25150.0,25028.0
1,Jessica,F,24192.0,9469.0
2,Ashley,F,23676.0,14370.0
3,Sarah,F,21029.0,12732.0
...,...,...,...,...
38403,Zymarion,M,NaN,5.0
38404,Zymeir,M,NaN,5.0
38405,Zyrell,M,NaN,5.0
38406,Zyron,M,NaN,5.0


Names like "Zyrell" and "Zyron" appeared in the 2004 data but not the 1996 data. For this reason, their count in 1996 is `NaN`. In general, there will be `NaN`s in a `DataFrame` resulting from an outer join. Any time a name appears in one `DataFrame` but not the other, there will be `NaN`s in the columns from the `DataFrame` whose data is missing.

In [7]:
names_outer.isnull().sum()

Name           0
Sex            0
Count_x    11987
Count_y     6367
dtype: int64

By contrast, there are no `NaN`s when we do an inner join. That is because we restrict to only the (name, sex) pairs that appeared in both `DataFrame`s, so we have counts for both 1996 and 2014.

In [8]:
names_inner.isnull().sum()

Name       0
Sex        0
Count_x    0
Count_y    0
dtype: int64

Left and right joins preserve data from one `DataFrame` but not the other. For example, if we were trying to calculate the percentage change for each name from 1996 to 2004, we would want to include all of the names that appeared in the 1996 data. If the name did not appear in the 2004 data, then that is informative.

In [9]:
# left join
names_left = names1996.merge(names2004, on=["Name", "Sex"], how="left")
names_left

,Name,Sex,Count_x,Count_y
0,Emily,F,25150,25028.0
1,Jessica,F,24192,9469.0
2,Ashley,F,23676,14370.0
3,Sarah,F,21029,12732.0
...,...,...,...,...
26416,Zildjian,M,5,NaN
26417,Zishe,M,5,NaN
26418,Zoran,M,5,14.0
26419,Zyler,M,5,23.0


The result of a left join has `NaN`s in the column from the right `DataFrame`.

In [10]:
names_left.isnull().sum()

Name          0
Sex           0
Count_x       0
Count_y    6367
dtype: int64

The result of a right join, on the other hand, has `NaN`s in the column from the left `DataFrame`.

In [11]:
# right join
names_right = names1996.merge(names2004, on=["Name", "Sex"], how="right")
names_right

,Name,Sex,Count_x,Count_y
0,Emily,F,25150.0,25028
1,Jessica,F,24192.0,9469
2,Ashley,F,23676.0,14370
3,Sarah,F,21029.0,12732
...,...,...,...,...
32036,Zymarion,M,NaN,5
32037,Zymeir,M,NaN,5
32038,Zyrell,M,NaN,5
32039,Zyron,M,NaN,5


In [12]:
names_right.isnull().sum()

Name           0
Sex            0
Count_x    11987
Count_y        0
dtype: int64

One way to visualize the different types of joins is using Venn diagrams. The shaded circles specify which values are included in the output.

![](joins.jpeg)

# Exercises

Exercises 1-2 deal with the Movielens data (`/data301/data/ml-1m/`) that you explored in the previous section.

In [24]:
movies_df = pd.read_csv("/data301/data/ml-1m/movies.dat",
            sep="::", 
            header=None, 
            names=["MovieID", "Title", "Genres"],
            engine="python")

ratings_df = pd.read_csv("/data301/data/ml-1m/ratings.dat",
            sep="::", 
            header=None, 
            names=["UserID", "MovieID", "Rating", "Timestamp"],
            engine="python")

users_df = pd.read_csv("/data301/data/ml-1m/users.dat",
            sep="::", 
            header=None, 
            names=["UserID", "Gender", "Age", "Occupation", "Zip-code"],
            engine="python")

**Exercise 1.** Calculate the number of ratings by movie. How many of the movies had zero ratings?

(_Hint_: Why is an inner join not sufficient here?)

In [44]:
movies_df.merge(ratings_df, on="MovieID", how="left").Rating.isna().sum()

177

**Exercise 2.** How many movies received both a 1 and a 5 rating? Do this by creating and joining two appropriate tables.

In [95]:
one = ratings_df.set_index("Rating").loc[1, :].set_index("MovieID")
five = ratings_df.set_index("Rating").loc[5, :].set_index("MovieID")
len(one.index.intersection(five.index).unique())
#this is the better way to do it, but oh whale

2986